##### Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, которая будет добавлять только новые вакансии/продукты в вашу базу.

In [15]:
from pprint import pprint
import requests
import json

from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError 

In [16]:
# подключаемся к клиенту Mongo на локальной машине
client = MongoClient()

client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [17]:
# cоздание датабазы vacancies

db = client['vacancies']
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'vacancies')

In [18]:
# импортируем из json файла список вакансий, которые получили на прошлом задании и сохранили в файл

with open('output3.json', 'r', encoding = 'utf-8') as f:
    vacancy_list = json.load(f)
    
# вывод для примера 3 первых вакансий
vacancy_list[:3]

[{'Старший дата-инженер / Senior Data Engineer': {'salary': 'от 270 000 руб.',
   'link': 'https://hh.ru/vacancy/72161518?query=data+engineer',
   'site': 'hh.ru'}},
 {'Дата-инженер / Data Engineer': {'salary': 'от 130 000 руб.',
   'link': 'https://hh.ru/vacancy/71236005?query=data+engineer',
   'site': 'hh.ru'}},
 {'Middle/Senior Data Engineer в Sber AI Lab': {'salary': 'ЗП не указана',
   'link': 'https://hh.ru/vacancy/73471713?query=data+engineer',
   'site': 'hh.ru'}}]

In [19]:
# создадим коллекию vacancies в нашей базе данных vacancies и сохраним в переменную vacancies_collection 

vacancies_collection = db.vacancies
vacancies_collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'vacancies'), 'vacancies')

In [20]:
# создадим функцию добавления вакансий в нашу коллекцию вакансий в базе данных

def add_vacancies_in_vacancies_collection(vacancy_list):
    
    # уникальным идентификатором вакансии будем считать ссылку (название вакансии не всегда оригинальное)
    # создадим список ссылок наших вакансий в коллекции
    list_links = []
    for doc in vacancies_collection.find():
        list_links.append(doc['link'])
    
    # проходим циклом по каждой вакансии и формируем документ для записи в коллекцию
    for vacancy in vacancy_list:
       
        # название вакансии
        title = list(vacancy.keys())[0]
        
        # создаем документ с ключом 'title' и значением - название вакансии
        document = {'title': title}
     
        # проходим циклом по остальным атрибутам вакансии и записываем их значения в соответсвующую графу
        for key in vacancy[title].keys():
            
            document[key] = vacancy[title][key]
        
        # обновляем наш документ в коллекции, если ссылка от него в списке ссылок наших вакансий list_links
        # иначе добавляем новый документ в коллекцию
        if document['link'] in list_links:
            vacancies_collection.replace_one({'link': document['link']}, document)
        else:
            vacancies_collection.insert_one(document)

In [21]:
# и запускаем функцию для добавления данных в коллекцию

add_vacancies_in_vacancies_collection(vacancy_list)

In [22]:
# количество записей коллекции
# если запустить функцию еще раз, то данные в коллекцие просто перезапишутся, так как ссылки на вакансии те же

len(list(vacancies_collection.find()))

500

In [23]:
# создадим список из новых вакансий для проверки нашей функции.
# тут для примера создали ссылки вручную, чтобы проверить, что функция вставляет новые значения.

new_vacancy = [
    
    {'Data engineer': {'salary': 'ЗП не указана',
   'link': 'https://rabota.ru/vacancy/73471184?query=data+engineer',
   'site': 'rabota.ru'}},
    
    {'Python developer': {'salary': 'ЗП не указана',
   'link': 'https://job.ru/vacancy/73468878?query=data+engineer',
   'site': 'job.ru'}}
    
]

In [24]:
# заново запускаем функцию для добавления данных в коллекцию

add_vacancies_in_vacancies_collection(new_vacancy)

In [25]:
# количество записей коллекции увеличилось на 2, значит новые вакансии добавлены

len(list(vacancies_collection.find()))

502

In [26]:
# выведем последние 3 вакансии из коллекции и посмотрим на наши новые вакансии
list(vacancies_collection.find()[499:])

[{'_id': ObjectId('63ac842c35781f10645165b3'),
  'title': 'Senior DBA / Администратор баз данных',
  'salary': 'ЗП не указана',
  'link': 'https://hh.ru/vacancy/73541299?query=data+engineer',
  'site': 'hh.ru'},
 {'_id': ObjectId('63ac842c35781f10645165b4'),
  'title': 'Data engineer',
  'salary': 'ЗП не указана',
  'link': 'https://rabota.ru/vacancy/73471184?query=data+engineer',
  'site': 'rabota.ru'},
 {'_id': ObjectId('63ac842c35781f10645165b5'),
  'title': 'Python developer',
  'salary': 'ЗП не указана',
  'link': 'https://job.ru/vacancy/73468878?query=data+engineer',
  'site': 'job.ru'}]

In [27]:
#vacancies_collection.delete_many({})